# 로지스틱 회귀(Logistic Regression)
### 이름은 회귀 이지만 분류모델, 이 알고리즘은 선형 회귀와 동일하게 선형 방정식을 학습.

z = a x (weight) + b x (length) + c x (diagonal) + d x (Height) + e x (width) + f

- a,b,c,d,e 는 계수
- 특성은 늘어났지만 다중회귀를 위한 선형 방정식
- z 는 확률이 되려면 0~1(0~100%)사이 값이 되어야 한다.
- z가 큰 음수일 때 0이되고, 큰 양수일 때 1이 되도록 바꾸는 방법이 로지스틱 함수(시그모이드 함수)를 사용하면 가능

In [2]:
import pandas as pd
fish = pd.read_csv('https://bit.ly/fish_csv_data')
fish.head()

,Species,Weight,Length,Diagonal,Height,Width
0,Bream,242.0,25.4,30.0,11.5200,4.0200
1,Bream,290.0,26.3,31.2,12.4800,4.3056
2,Bream,340.0,26.5,31.1,12.3778,4.6961
3,Bream,363.0,29.0,33.5,12.7300,4.4555
4,Bream,430.0,29.0,34.0,12.4440,5.1340


In [3]:
print(pd.unique(fish['Species']))

['Bream' 'Roach' 'Whitefish' 'Parkki' 'Perch' 'Pike' 'Smelt']


In [4]:
fish_input = fish[['Weight', 'Length', 'Diagonal', 'Height', 'Width']].to_numpy()

In [5]:
print(fish_input[:5])

[[242.      25.4     30.      11.52     4.02  ]
 [290.      26.3     31.2     12.48     4.3056]
 [340.      26.5     31.1     12.3778   4.6961]
 [363.      29.      33.5     12.73     4.4555]
 [430.      29.      34.      12.444    5.134 ]]


In [6]:
fish_target = fish['Species'].to_numpy()

In [7]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(
    fish_input, fish_target, random_state=42
)

In [8]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(train_input)
train_scaled = ss.transform(train_input)
test_scaled = ss.transform(test_input)

In [9]:
from sklearn.neighbors import KNeighborsClassifier
kn = KNeighborsClassifier(n_neighbors=3)
kn.fit(train_scaled, train_target)
print(kn.score(train_scaled, train_target))
print(kn.score(test_scaled, test_target))

0.8907563025210085
0.85


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
z = np.arange(-5,5,0.1)
phi = 1/(1+np.exp(-z))
plt.plot(z, phi)
plt.xlabel('z')
plt.ylabel('phi')
plt.show()

2진 분류일 경우 시그모이드 함수의 출력이 0.5보다 크면 양수 클래스, 0.5보다 작거나 같으면 음성 클래스로 판단

In [10]:
bream_smelt_indexes = (train_target=='Bream')|(train_target=='Smelt')
train_bream_smelt = train_scaled[bream_smelt_indexes]
target_bream_smelt = train_target[bream_smelt_indexes]

In [11]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(train_bream_smelt, target_bream_smelt)

LogisticRegression()

In [12]:
print(lr.predict(train_bream_smelt[:5]))
# 훈련한 모델을 사용해 처음 5개 샘플을 예측

['Bream' 'Smelt' 'Bream' 'Bream' 'Bream']


In [14]:
print(lr.classes_)
print(lr.predict_proba(train_bream_smelt[:5]))
# 예측 확률 predict_proba() 메소드에서 제공

['Bream' 'Smelt']
[[0.99759855 0.00240145]
 [0.02735183 0.97264817]
 [0.99486072 0.00513928]
 [0.98584202 0.01415798]
 [0.99767269 0.00232731]]


In [16]:
print(lr.coef_, lr.intercept_)
# 로지스틱 회귀가 학습한 계수

[[-0.4037798  -0.57620209 -0.66280298 -1.01290277 -0.73168947]] [-2.16155132]


로지스틱 회귀 모델이 학습한 방정식 =

z = -0.404 x (weight) - 0.576 x (length) - 0.663 x (diagonal) - 1.013 x (height) - 0.732 x (width) - 2.161


In [17]:
# LogisticRegression모델로 z값 계산하기
decisions = lr.decision_function(train_bream_smelt[:5])
print(decisions)

[-6.02927744  3.57123907 -5.26568906 -4.24321775 -6.0607117 ]


In [18]:
# z 값을 시그모이드 함수에 통과시키면 확률을 얻을 수 있다.
from scipy.special import expit
print(expit(decisions))

[0.00240145 0.97264817 0.00513928 0.01415798 0.00232731]


여기서 얻을 수 있는 결론 :

decision_function() 메소드는 양성클래스에 대한 z값 반환

이진 분류일 경우

predict_proba(): 음성클래스와 양성클래스에 대한 확률 출력
decision_function(): 양성클래스에 대한 z값 계산
coef_, intercept_ 속성 : 로지스틱 모델이 학습한 선형 방정식의 계수가 들어있다.

## 로지스틱 회귀로 다중 분류 수행

LogisticRegression에서 규제를 제어하는 매개변수는 C

C는 alpha와 반대로 커질수록 규제가 작아진다.

In [19]:
lr = LogisticRegression(C=20, max_iter=1000)
lr.fit(train_scaled, train_target)
print(lr.score(train_scaled, train_target))
print(lr.score(test_scaled, test_target))

0.9327731092436975
0.925


In [20]:
print(lr.predict(test_scaled[:5]))

['Perch' 'Smelt' 'Pike' 'Roach' 'Perch']


In [21]:
proba = lr.predict_proba(test_scaled[:5])
print(np.round(proba, decimals=3))
# 1번째 샘플은 Perch일 확률이 84%, 2번째 샘플은 Smelt일 확률이 약 95%

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]


In [22]:
print(lr.classes_)

['Bream' 'Parkki' 'Perch' 'Pike' 'Roach' 'Smelt' 'Whitefish']


Softmax함수란?

- 여러개의 선형 방정식의 출력값을 0~1 사이로 압축하고 전체 합이 1이되도록 만든다.
- 7개의 z값을 (z1~z7) 이라고 하면

e_sum = e^z1 + e^z2 + e^z3 + e^z4 + e^z5 + e^z6 + e^z7

s1 = e^z1/e_sum, ... s7 = e^z7/e_sum 이고

s1 + s2 + s3 + s4 +s5 + s6 +s7 을 하면 분자와 분모가 같아서 1이된다.

7개 생선에 대한 확률의 합은 1이 되어야한다.

In [24]:
 # 5개샘플에 대한 z1~z7의 값 구한다.
decision = lr.decision_function(test_scaled[:5])
print(np.round(decision, decimals=2))

[[ -6.5    1.03   5.16  -2.73   3.34   0.33  -0.63]
 [-10.86   1.93   4.77  -2.4    2.98   7.84  -4.26]
 [ -4.34  -6.23   3.17   6.49   2.36   2.42  -3.87]
 [ -0.68   0.45   2.65  -1.19   3.26  -5.75   1.26]
 [ -6.4   -1.99   5.82  -0.11   3.5   -0.11  -0.71]]


In [25]:
# 소프트맥스 함수로 모든 z값들을 0~1사이로 압축한다
from scipy.special import softmax
proba = softmax(decision, axis=1)
print(np.round(proba, decimals=3))

[[0.    0.014 0.841 0.    0.136 0.007 0.003]
 [0.    0.003 0.044 0.    0.007 0.946 0.   ]
 [0.    0.    0.034 0.935 0.015 0.016 0.   ]
 [0.011 0.034 0.306 0.007 0.567 0.    0.076]
 [0.    0.    0.904 0.002 0.089 0.002 0.001]]
